In [ ]:
!pip install pyspark
!pip3 install pyspark
!pip install findspark
!pip3 install findspark

In [ ]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
df = spark.read.format("csv") \
.option("header", "true") \
.option("nullValue","NA") \
.option("inferSchema", "true").load("2007.csv")

In [ ]:
df.printSchema()

In [ ]:
print("Partitions:", df.rdd.getNumPartitions())

In [ ]:
#Eliminar columnes
df2 = df.drop("Flightfrom pyspark.sql.functions import exprNum","TailNum","UniqueCarrier")
#New frame with columns
df2 = df.select("Origin", "Dest", "ArrDelay", "DepDelay")
df2.show()
#Eliminar files on hi ha NA
df3 = df2.na.drop()

In [ ]:
from pyspark.sql.functions import expr
##Suma, expr we can do expressions
df4 = df3.withColumn("SumDelay", expr("ArrDelay + DepDelay"))
#Action, no transformation
df4.select("DepDelay", "ArrDelay", "SumDelay").show(10)

In [ ]:
from pyspark.sql.functions import max, min
#es recorre tot el df
df4.select(max("SumDelay"),min("SumDelay")).show()

In [ ]:
from pyspark.sql.functions import avg
df4.select(max("SumDelay"),min("SumDelay"),avg("SumDelay")).show()

In [ ]:
#temporal save in cache or ram
df4.cache()

In [ ]:
df5 = df4.where("SumDelay < 0")
df5.show() #accio, nomes mostra els primer elements
df3.count()
df5.count()

In [ ]:
df5 = df4.where("SumDelay < 0").where("Origin == ’JFK’")
df5.show()
df5.count()

In [ ]:
#L’altre funció de filtratge disponible es diu filter. És molt similar a la funció where, amb l’a-
#vantatge que es poden fer servir varibles de Python per establir els criteris sobre les condicions de
#filtratge.
from pyspark.sql.functions import col
i = 0
city = "JFK"
df5 = df4.filter(col("SumDelay") < i).filter(col("Origin") == city)
df5.count()

In [ ]:
#Exercici
from pyspark.sql.functions import sum

In [ ]:
from pyspark.sql.functions import asc, desc
df5 = df4.sort(asc("SumDelay"))
df5.show()
df5 = df4.sort(desc("SumDelay")).limit(5)
df5.show()

In [ ]:
#UNIQUE Elements
df5 = df4.select("Origin").distinct()
df5.count(

In [ ]:
#Exercici
#Quantes destinacions diferents hi ha? A més, què és el que fa la següent operació?
df4.select("Origin","Dest").distinct().count()

In [ ]:
dades = df4.limit(5).collect()
dades
dades[0]
dades[0][3]

In [ ]:
#Escriure a disc
df4.rdd.getNumPartitions()
#Per defecte, si ara escrivíssim les dades a disc, es crearien tants fitxers com particions hi ha
df4_one = df4.coalesce(1)
df4_one.rdd.getNumPartitions()
df4_one = spark.write.format("csv") \
.option("header", "true") \
.save("df4_one.csv")

In [ ]:
##Exercici
"""És interessant que feu un experiment: abans hem ordenat els elements de df4 pel valor de SumDelay.
Intenteu ordenar ara els valors de df4_one. Hi haurà alguna diferència en l’eficiència de l’ordenació?
Feu servir alguna eina gràfica que us mostri el consum de CPU per veure-ho. Per a què és útil tenir
les dades dividides en particions?
"""
